In [43]:
import os
import csv
import glob
import MySQLdb
import scrapy
from selenium import webdriver
from scrapy.selector import Selector
from scrapy.http import Request
from time import sleep
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from sqlalchemy import *
import numpy as np
import mysql.connector
import lxml
from lxml import etree as ElementTree

In [44]:
def process(things):
    df=pd.DataFrame(data={'author':[],'heat':[]})
    for i in range(len(things)):
        listing=things[i]
        path = lxml.html.fromstring(listing)
        heat=path.xpath('//li[@class="pageviews"]//text()')
        a=0
        for i in range(len(heat)):
            b=i-a
            if heat[b][0].isdigit()==False:
                del heat[b]
                a=a+1
            else:
                heat[b]=heat[b].replace(',','')
        if len(heat)==0:
            heat=None
        else:
            heat=int(heat[0])
        authors=path.xpath('//li[@class="author-byline ks-author-byline"]//text()')
        a=0
        for i in range(len(authors)):
            b=i-a
            authors[b]=authors[b].replace(' ','')
            authors[b]=authors[b].replace('and','')
            authors[b]=authors[b].replace(',','')
            if authors[b]=='':
                del authors[b]
                a=a+1
        for author in authors:
            df.set_value(len(df), 'author', author)
            df.set_value(len(df)-1, 'heat', heat)
    return df

In [45]:
driver=webdriver.Chrome('C://Users/USER/Desktop/sougou/chromedriver')
driver.get('http://www.businessinsider.com/clusterstock?page=1')
df=pd.DataFrame(data={'author':[],'heat':[]})
while True:
    try:
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            sleep(0.5)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        sleep(2)
        sel = Selector(text=driver.page_source)
        things=sel.xpath('//div[@class="flex byline"]//ul').extract()
        df_new=process(things)
        df=pd.concat([df,df_new])
        next_page = driver.find_element_by_xpath('//a[@class="float-right btn button-white"]')
        next_page.click()
        sleep(2)
    except NoSuchElementException:
        driver.quit()
        break

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [46]:
df

,author,heat
0,MattTurner,19465.0
1,AlexMorrell,2588.0
2,JoeCiolli,2366.0
3,JoePerticone,15.0
4,BobBryan,15.0
5,Reuters,0.0
6,Reuters,3.0
7,BeckyPeterson,127.0
8,GinaHeeb,NaN
9,BenjaminZhang,492.0


In [52]:
df.groupby(['author']).count()

,heat
author,
AkinOyedele,13
AlanDawson,1
AlexMorrell,10
AlexriaSage,2
AlisonMillington,2
AllStarCharts,1
AllanSmith,1
AnaelePelisson,1
AndrewMeola,0


In [48]:
mean=df.groupby(['author']).mean()

In [53]:
count=df.groupby(['author']).count()

In [54]:
data=pd.merge(count,mean, left_on='author', right_on='author')
data.to_csv('data.csv')